In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

# cf.) shp 파일 csv 파일로 가공하는 방법

In [ ]:
sp = gpd.read_file('C:/빅캠공모전/data/2014년하/TL_SPBD_BULD.shp')

In [ ]:
sp.iloc[:,0:37].head()

In [ ]:
sp.iloc[:,0:37].to_csv('2014_shp.csv',encoding="euc-kr")

In [ ]:
sp_2 = gpd.read_file('C:/빅캠공모전/data/2015년상/TL_SPBD_BULD.shp')

In [ ]:
sp_2.iloc[:,0:36].to_csv('2015_shp.csv',encoding="utf-8")

In [ ]:
sp_3 = gpd.read_file('C:/빅캠공모전/data/2016년상/TL_SPBD_BULD.shp')

In [ ]:
sp_3.iloc[:,0:36].to_csv('2016_shp.csv',encoding="euc-kr")

# 전처리 목록

A. shp 파일에서 고층(6층 이상)/초고층(20층 이상) 건물 뽑아 가공하여 shp파일로 저장
B. X,Y 값을 위경도 값으로 반환하여 저장

In [2]:
# 데이터 확인
data = gpd.read_file("../data/2014_building/2016/TL_SPBD_BULD.shp",encoding="euc-kr")
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 654595 entries, 0 to 654594
Data columns (total 37 columns):
BDTYP_CD      654595 non-null object
BD_MGT_SN     654595 non-null object
BSI_INT_SN    654595 non-null float64
BSI_ZON_NO    654595 non-null object
BULD_MNNM     654595 non-null int64
BULD_NM       115369 non-null object
BULD_NM_DC    50664 non-null object
BULD_SE_CD    654595 non-null object
BULD_SLNO     654595 non-null int64
BUL_DPN_SE    654595 non-null object
BUL_MAN_NO    654595 non-null int64
EMD_CD        654595 non-null object
GRO_FLO_CO    654595 non-null int64
LI_CD         654595 non-null object
LNBR_MNNM     654595 non-null int64
LNBR_SLNO     654595 non-null int64
MNTN_YN       654595 non-null object
RN_CD         654595 non-null object
SIG_CD        654595 non-null object
UND_FLO_CO    654595 non-null int64
ZIP           0 non-null object
X_AXIS        654595 non-null int64
Y_AXIS        654595 non-null int64
HOUS_ID       654595 non-null object
TOTAL_A

# A. 고층/초고층 빌딩 뽑아내기

In [4]:
# 6층 이상 빌딩 : 고층빌딩으로 저장
high_bld = data[data.GRO_FLO_CO >= 6]
print("고층빌딩 데이터 수 : ",len(high_bld))

# 20층 이상 빌딩 : 초고층빌딩으로 저장
ext_high_bld = data[data.GRO_FLO_CO >= 20]
print("초고층빌딩 데이터 수 :",len(ext_high_bld))

# 각 정보 저장하기
# high_bld.to_file("../data/buildings/high_buildings_over6.shp",encoding="euc-kr")
# ext_high_bld.to_file("../data/buildings/extreme_high_buildings_over20.shp",encoding="euc-kr")

고층빌딩 데이터 수 :  40797
초고층빌딩 데이터 수 : 4542


# B. X,Y 값 위경도 값으로 변환

In [6]:
high_bld = high_bld.reset_index()
del high_bld["index"]
high_bld.columns

Index(['BDTYP_CD', 'BD_MGT_SN', 'BSI_INT_SN', 'BSI_ZON_NO', 'BULD_MNNM',
       'BULD_NM', 'BULD_NM_DC', 'BULD_SE_CD', 'BULD_SLNO', 'BUL_DPN_SE',
       'BUL_MAN_NO', 'EMD_CD', 'GRO_FLO_CO', 'LI_CD', 'LNBR_MNNM', 'LNBR_SLNO',
       'MNTN_YN', 'RN_CD', 'SIG_CD', 'UND_FLO_CO', 'ZIP', 'X_AXIS', 'Y_AXIS',
       'HOUS_ID', 'TOTAL_AREA', 'ADMI_CD', 'BLK_CD', 'OA_CD', 'FL_UP_NEW',
       'FL_DN_NEW', 'USE_CD_NEW', 'XMIN', 'YMIN', 'XMAX', 'YMAX', 'SHAPE_AREA',
       'geometry'],
      dtype='object')

In [7]:
from pyproj import Proj, transform

# Projection 정의
# EPSG 5181
proj_EPSG = Proj(init='epsg:5181') # 한국형 좌표계
# WGS 84
proj_WGS = Proj(init='epsg:4326') # GPS사용 전지구 좌표

# EPSG 5181 -> WGS 84
x_list = []
y_list = []
x_max_list = []
y_max_list = []
x_min_list = []
y_min_list = []
for n in range(0,len(high_bld.index)) :
    x = high_bld.X_AXIS[n]
    y = high_bld.Y_AXIS[n]
    x_max = high_bld.XMAX[n]
    y_max = high_bld.YMAX[n]
    x_min = high_bld.XMIN[n]
    y_min = high_bld.YMIN[n]
    
    x_new, y_new = transform(proj_EPSG, proj_WGS, x, y)
    x_mx_new, y_mx_new = transform(proj_EPSG, proj_WGS, x_max, y_max)
    x_mn_new, y_mn_new = transform(proj_EPSG, proj_WGS, x_min, y_min)
    
    x_list.append(x_new)
    y_list.append(y_new)
    x_max_list.append(x_mx_new)
    y_max_list.append(y_mx_new)
    x_min_list.append(x_mn_new)
    y_min_list.append(y_mn_new)

In [8]:
# 좌표 변환 내용 확인
print("변환된 x좌표 : ", x_list[0:10], "...")
print("변환된 y좌표 : ", y_list[0:10], "...")

변환된 x좌표 :  [126.9874012202516, 126.9776552721847, 126.97631905287506, 126.97574159005306, 126.97617178399923, 126.97783539920908, 126.97829949911758, 126.97780121984803, 126.97818606228334, 126.97520858710062] ...
변환된 y좌표 :  [37.5707988510621, 37.569869383494456, 37.571499924507684, 37.571977334680746, 37.57185128298034, 37.57319408961611, 37.573266255431584, 37.57393289897596, 37.5740771298813, 37.574905457282604] ...


In [9]:
# 데이터 프레임에 추가하기
high_bld["lat_center"] = pd.DataFrame(y_list)
high_bld["lon_center"] = pd.DataFrame(x_list)
high_bld["lat_max"] = pd.DataFrame(y_max_list)
high_bld["lon_max"] = pd.DataFrame(x_max_list) 
high_bld["lat_min"] = pd.DataFrame(y_min_list) 
high_bld["lon_min"] = pd.DataFrame(x_min_list) 

#확인
high_bld.head()

,BDTYP_CD,BD_MGT_SN,BSI_INT_SN,BSI_ZON_NO,BULD_MNNM,BULD_NM,BULD_NM_DC,BULD_SE_CD,BULD_SLNO,BUL_DPN_SE,...,XMAX,YMAX,SHAPE_AREA,geometry,lat_center,lon_center,lat_max,lon_max,lat_min,lon_min
0,04999,1111013600100750001031868,16388.0,03164,405,태영빌딩,None,0,0,M,...,198891,452372,99.92,"POLYGON ((198890.769 452356.798, 198882.411 45...",37.570799,126.987401,37.570889,126.987446,37.570736,126.987333
1,10203,1111011900101390008017955,13091.0,03187,152,일민미술관,None,0,0,M,...,198035,452277,563.59,"POLYGON ((198024.977 452242.470, 198024.924 45...",37.569869,126.977655,37.570032,126.977757,37.569689,126.977553
2,06999,1111011900101570004028349,13100.0,03183,169,None,None,0,0,M,...,197912,452448,119.92,"POLYGON ((197911.915 452447.747, 197911.966 45...",37.571500,126.976319,37.571572,126.976364,37.571419,126.976251
3,10101,1111011900100810003027702,13103.0,03172,175,None,세종문화회관,0,0,M,...,197882,452528,2412.80,"POLYGON ((197870.363 452527.038, 197871.111 45...",37.571977,126.975742,37.572293,126.976024,37.571698,126.975515
4,10101,1111011900100810003027269,13103.0,03172,175,None,세종문화회관,0,0,M,...,197910,452497,848.10,"POLYGON ((197898.141 452496.288, 197898.291 45...",37.571851,126.976172,37.572013,126.976342,37.571698,126.976002


In [ ]:
# 고층 빌딩 변환값 저장
high_bld.to_file("../data/buildings/high_bld_over6_include_latlon.shp",encoding="euc-kr")

# 초고층 빌딩 변환값 저장
high_bld[high_bld.GRO_FLO_CO >= 20].to_file("../data/buildings/high_bld_over20_include_latlon.shp",encoding="euc-kr")